In [ ]:
/* Proposition 1 — Orders on the Last Active Day
This query identifies all orders made on the final day of business activity. It’s useful for verifying end-of-period transactions and checking data completeness. It demonstrates critical thinking and technical accuracy, using a scalar subquery to isolate the latest date.
*/

SELECT i.CustomerID, i.InvoiceID, i.InvoiceDate
FROM Sales.Invoices AS i
WHERE i.InvoiceDate = (
    SELECT MAX(i2.InvoiceDate)
    FROM Sales.Invoices AS i2
    WHERE i2.CustomerID = i.CustomerID
)
ORDER BY i.CustomerID, i.InvoiceID;

/* Proposition 2 — Orders by the Top Customer
This query finds the customer with the highest number of orders and lists all of their transactions. It’s ideal for identifying loyal or high-revenue clients. It represents communication and analytical reasoning, as it transforms SQL output into clear customer insights.
*/

SELECT DISTINCT i.CustomerID
FROM Sales.Invoices AS i
WHERE i.InvoiceID IN (
    SELECT il.InvoiceID
    FROM Sales.InvoiceLines AS il
    WHERE ISNULL(il.UnitPrice, 0) > 100
)
ORDER BY i.CustomerID;

/* Proposition 3 — Employees with No Recent Orders
This query lists employees who haven’t processed any orders since May 1st, 2016. It’s helpful for identifying inactive employees or potential training gaps. It uses a NOT EXISTS subquery and reflects professionalism and critical evaluation.
*/

SELECT s.SupplierID, s.SupplierName
FROM Purchasing.Suppliers AS s
WHERE NOT EXISTS (
    SELECT 1
    FROM Warehouse.StockItems AS si
    WHERE si.SupplierID = s.SupplierID
)
ORDER BY s.SupplierID;

/* Proposition 4 — Countries with Customers but No Employees
This query finds countries where customers exist but employees don’t. It reveals potential expansion opportunities or support gaps in certain regions. It highlights strategic thinking and data comparison, using subqueries to analyze country coverage differences.
*/

SELECT
    il.InvoiceID,
    il.InvoiceLineID,
    il.ExtendedPrice,
    CAST(
        100.0 * il.ExtendedPrice /
        NULLIF((
            SELECT SUM(x.ExtendedPrice)
            FROM Sales.InvoiceLines AS x
            WHERE x.InvoiceID = il.InvoiceID
        ), 0) AS DECIMAL(10,2)
    ) AS PctOfInvoice
FROM Sales.InvoiceLines AS il
ORDER BY il.InvoiceID, il.InvoiceLineID;

/* Proposition 5 — Customer’s Last Active Day
This query returns all orders made by each customer on their most recent order date. It’s a way to analyze recency and identify active versus dormant customers. It shows technical proficiency and analytical thought, using correlated subqueries to pinpoint each customer’s latest activity.
*/

SELECT TOP (10) d.OrderID, d.TotalQty
FROM (
    SELECT ol.OrderID, SUM(ol.Quantity) AS TotalQty
    FROM Sales.OrderLines AS ol
    GROUP BY ol.OrderID
) AS d
ORDER BY d.TotalQty DESC, d.OrderID;

/* Proposition 6 — Customers Active in 2015 but Not 2016
This query identifies customers who placed orders in 2015 but did not return in 2016. It’s used to detect churn and evaluate customer retention. It demonstrates critical thinking and communication, connecting data analysis with customer relationship management.
*/

WITH M AS (
    SELECT
        DATEFROMPARTS(YEAR(i.InvoiceDate), MONTH(i.InvoiceDate), 1) AS InvoiceMonth,
        SUM(il.ExtendedPrice) AS MonthSales
    FROM Sales.Invoices AS i
    JOIN Sales.InvoiceLines AS il ON il.InvoiceID = i.InvoiceID
    GROUP BY DATEFROMPARTS(YEAR(i.InvoiceDate), MONTH(i.InvoiceDate), 1)
)
SELECT InvoiceMonth, MonthSales
FROM M
ORDER BY InvoiceMonth;

/* Proposition 7 — Customers Who Bought a Specific Product
This query lists customers who purchased product 12. It’s simple but valuable for targeted marketing or identifying cross-sell opportunities. It applies technical accuracy and business awareness, showing SQL’s role in supporting marketing strategies.
*/

SELECT i.InvoiceID, a.InvoiceLineID, a.StockItemID, a.ExtendedPrice
FROM Sales.Invoices AS i
CROSS APPLY (
    SELECT TOP (2) il.InvoiceLineID, il.StockItemID, il.ExtendedPrice
    FROM Sales.InvoiceLines AS il
    WHERE il.InvoiceID = i.InvoiceID
    ORDER BY il.ExtendedPrice DESC, il.InvoiceLineID
) AS a
ORDER BY i.InvoiceID, a.InvoiceLineID;

/* Proposition 8 — Running Total of Quantities per Customer
This query calculates a running total of ordered quantities per month for each customer. It helps track cumulative demand over time without relying on window functions. It demonstrates critical thinking and technical fluency, using correlated subqueries for progressive aggregation.
*/

SELECT
    i.CustomerID,
    i.InvoiceID,
    (SELECT SUM(x.ExtendedPrice)
     FROM Sales.InvoiceLines AS x
     WHERE x.InvoiceID = i.InvoiceID) AS InvoiceTotal
FROM Sales.Invoices AS i
WHERE (SELECT SUM(x.ExtendedPrice)
       FROM Sales.InvoiceLines AS x
       WHERE x.InvoiceID = i.InvoiceID)
      =
      (SELECT MAX(T.InvoiceTotal)
       FROM (
           SELECT i2.InvoiceID, SUM(il2.ExtendedPrice) AS InvoiceTotal
           FROM Sales.Invoices AS i2
           JOIN Sales.InvoiceLines AS il2 ON il2.InvoiceID = i2.InvoiceID
           WHERE i2.CustomerID = i.CustomerID
           GROUP BY i2.InvoiceID
       ) AS T)
ORDER BY i.CustomerID, i.InvoiceID;

/* Proposition 9 — Days Between Consecutive Orders
This query calculates the gap in days between consecutive orders for each customer. It helps understand purchase frequency and predict future order patterns. It represents analytical skill and professionalism, translating data patterns into actionable insight.
*/

SELECT DISTINCT i.CustomerID
FROM Sales.Invoices AS i
WHERE EXISTS (
    SELECT 1
    FROM Sales.InvoiceLines AS il
    JOIN Warehouse.StockItems AS si ON si.StockItemID = il.StockItemID
    WHERE il.InvoiceID = i.InvoiceID
      AND si.IsChillerStock = 1
)
ORDER BY i.CustomerID;

/* Proposition 10 — Yearly Totals by Employee
This query builds a view showing total quantities handled per employee each year and includes running totals for performance comparison. It supports annual evaluations and workload planning. It reflects technical proficiency, teamwork, and leadership, demonstrating the ability to organize reusable analytical structures.
*/

WITH Y AS (
    SELECT
        i.SalespersonPersonID,
        YEAR(i.InvoiceDate) AS OrderYear,
        COUNT(DISTINCT i.InvoiceID) AS NumInvoices
    FROM Sales.Invoices AS i
    GROUP BY i.SalespersonPersonID, YEAR(i.InvoiceDate)
)
SELECT SalespersonPersonID, OrderYear, NumInvoices
FROM Y
ORDER BY SalespersonPersonID, OrderYear;
